Célula 1: Imports e Configuração Inicial

In [1]:
# ARQUIVO: etapa_03.ipynb
# Imports de bibliotecas padrão do Python
import os
import shutil
import glob
import time
from contextlib import redirect_stdout
import traceback

# Import de bibliotecas de terceiros para análise de dados e visualização
import pandas as pd

# Imports das nossas classes e funções customizadas dos outros módulos do projeto
from instancia import parse_instance
from grafo import CustomGraph
from solucao import Solucao, Rota, preparar_servicos, construtivo_guloso_vizinho_mais_proximo
from otimizacao import swap_entre_rotas

print("Módulos carregados com sucesso.")



Módulos carregados com sucesso.


Célula 2: Processamento em Lote de Todas as Instâncias

In [2]:
# Definição dos diretórios de entrada (com as instâncias) e saída (para as soluções).
INPUT_DIR = 'dados/MCGRP'
OUTPUT_DIR = 'solucoes/'

# Garante que o diretório de soluções esteja limpo antes de uma nova execução,
# removendo a pasta e recriando-a para evitar o acúmulo de resultados antigos.
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Itera sobre todos os ficheiros de instância (.dat) no diretório de entrada.
# O 'sorted()' garante que os ficheiros sejam processados em ordem alfabética para consistência.
for filepath in sorted(glob.glob(os.path.join(INPUT_DIR, '*.dat'))):
    
    # Extrai o nome da instância a partir do nome do ficheiro para usar nos logs.
    inst_name = os.path.splitext(os.path.basename(filepath))[0]
    out_path = os.path.join(OUTPUT_DIR, f'sol-{inst_name}.dat')
    
    # Imprime um log no console para acompanhamento do progresso em tempo real.
    print(f'▶ Processando {inst_name}...')

    # Utiliza um bloco try-except para garantir que, se uma instância falhar,
    # o processamento geral não seja interrompido e continue para as próximas.
    try:
        # Abre o ficheiro de solução para escrita ('w').
        with open(out_path, 'w') as fout:
            # Redireciona toda a saída padrão (qualquer comando 'print') para o ficheiro de solução.
            # Isto garante que a saída siga estritamente o formato de entrega.
            with redirect_stdout(fout):

                # --- ETAPA 1: PARSING, GRAFO E CÁLCULO DE CAMINHOS MÍNIMOS ---
                
                # Regista o tempo de início do processamento para esta instância.
                tempo_inicio_total = time.perf_counter()
                
                # Lê e interpreta o ficheiro da instância.
                parsed_data = parse_instance(filepath)
                
                # Cria o objeto do grafo com base nos dados lidos.
                g = CustomGraph(parsed_data)

                # Calcula a matriz de caminhos mínimos para todos os pares de nós.
                dist, pred, node_to_index, index_to_node = g.all_pairs_dijkstra()

                # Prepara uma estrutura de dados unificada para acesso às informações dos serviços.
                servicos_info = preparar_servicos(parsed_data)

                # --- ETAPA 2: GERAÇÃO DA SOLUÇÃO INICIAL (CONSTRUTIVA) ---
                
                # Executa o algoritmo construtivo para gerar uma solução inicial válida.
                solucao_construtiva = construtivo_guloso_vizinho_mais_proximo(
                    g, servicos_info, dist, node_to_index
                )

                # --- ETAPA 3: OTIMIZAÇÃO DA SOLUÇÃO (MELHORIA) ---
                
                # Regista o tempo de início da fase de otimização.
                tempo_inicio_melhoria = time.perf_counter()

                # Aplica o algoritmo de busca local para melhorar a solução inicial.
                solucao_melhorada = swap_entre_rotas(
                    solucao_construtiva, g, servicos_info, dist, node_to_index
                )
                
                # Regista o tempo de término da fase de otimização.
                tempo_fim_melhoria = time.perf_counter()

                # --- FINALIZAÇÃO E ESCRITA DA SOLUÇÃO ---
                
                # Calcula os tempos de execução em microssegundos.
                clocks_total = (tempo_fim_melhoria - tempo_inicio_total) * 1_000_000
                clocks_melhor_sol = (tempo_fim_melhoria - tempo_inicio_melhoria) * 1_000_000

                # Chama a função que formata e imprime a solução final no ficheiro de saída.
                solucao_melhorada.print_formatado(
                    nome_instancia=inst_name,
                    tempo_total=clocks_total,
                    tempo_melhor_sol=clocks_melhor_sol
                )

        # Imprime uma mensagem de sucesso no console para feedback visual.
        print(f'✅ OK: {inst_name} salvo em {out_path}')

    except Exception as e:
        # Em caso de erro, imprime a exceção no console para depuração.
        print(f'❌ ERRO AO PROCESSAR {inst_name}: {e.__class__.__name__}')
        traceback.print_exc()

# Log final no console indicando que todos os ficheiros foram processados.
print("\nProcessamento de todas as instâncias concluído.")


▶ Processando BHW1...
✅ OK: BHW1 salvo em solucoes/sol-BHW1.dat
▶ Processando BHW10...
✅ OK: BHW10 salvo em solucoes/sol-BHW10.dat
▶ Processando BHW11...
✅ OK: BHW11 salvo em solucoes/sol-BHW11.dat
▶ Processando BHW12...
✅ OK: BHW12 salvo em solucoes/sol-BHW12.dat
▶ Processando BHW13...
✅ OK: BHW13 salvo em solucoes/sol-BHW13.dat
▶ Processando BHW14...
✅ OK: BHW14 salvo em solucoes/sol-BHW14.dat
▶ Processando BHW15...
✅ OK: BHW15 salvo em solucoes/sol-BHW15.dat
▶ Processando BHW16...
✅ OK: BHW16 salvo em solucoes/sol-BHW16.dat
▶ Processando BHW17...
✅ OK: BHW17 salvo em solucoes/sol-BHW17.dat
▶ Processando BHW18...
✅ OK: BHW18 salvo em solucoes/sol-BHW18.dat
▶ Processando BHW19...
✅ OK: BHW19 salvo em solucoes/sol-BHW19.dat
▶ Processando BHW2...
✅ OK: BHW2 salvo em solucoes/sol-BHW2.dat
▶ Processando BHW20...
✅ OK: BHW20 salvo em solucoes/sol-BHW20.dat
▶ Processando BHW3...
✅ OK: BHW3 salvo em solucoes/sol-BHW3.dat
▶ Processando BHW4...
✅ OK: BHW4 salvo em solucoes/sol-BHW4.dat
▶ Proce